<a href="https://colab.research.google.com/github/OsvaldoUfla/Atividade-com-GPU---05-08-2024/blob/main/Atividade_com_GPU_05_08_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/lesc-ufv/cad4u.git &> /dev/null
!git clone https://github.com/lesc-ufv/cad4u &> /dev/null
%load_ext plugin

In [13]:
%%writefile hellocuda.cu

#include <stdio.h>

__global__ void helloFromGPU()
{
    printf("Hello World from GPU! Block %d  Thread %d\n", blockIdx.x, threadIdx.x);
}

int main(int argc, char **argv)
{
    printf("Hello World from CPU!\n");

    helloFromGPU<<<1, 10>>>();
    cudaDeviceReset();

    cudaDeviceSynchronize();

    return 0;
}


Overwriting hellocuda.cu


In [14]:
!nvcc hellocuda.cu -o hellocuda
!./hellocuda

Hello World from CPU!
Hello World from GPU! Block 0  Thread 0
Hello World from GPU! Block 0  Thread 1
Hello World from GPU! Block 0  Thread 2
Hello World from GPU! Block 0  Thread 3
Hello World from GPU! Block 0  Thread 4
Hello World from GPU! Block 0  Thread 5
Hello World from GPU! Block 0  Thread 6
Hello World from GPU! Block 0  Thread 7
Hello World from GPU! Block 0  Thread 8
Hello World from GPU! Block 0  Thread 9


Atividade de soma de matrizes

In [15]:
%%gpu
#include <stdio.h>
#include <assert.h>

#define N 4

__global__ void additionMatricesKernel(int *d_a, int *d_b, int *d_c){
  //Define os índices das threads
  int i = threadIdx.y+blockIdx.y*blockDim.y;
  int j = threadIdx.x+blockIdx.x*blockDim.x;
  //Realiza a soma de cada elemento
  d_c[i*N+j] = d_a[i*N+j] + d_b[i*N+j];
}

int main(){
  //Define as matrizes quadradas NxN
    int h_a[N][N], h_b[N][N], h_c[N][N];

  //Preenche as matrizes
    for(int i = 0; i < N; i++){
        for(int j = 0; j < N; j++){
            h_a[i][j] = h_b[i][j] = i+j;
            h_c[i][j] = 0;
        }
    }

    //Declara os ponteiros para a GPU
    int *d_a, *d_b, *d_c;

    const int nBytes = N * N * sizeof(int);

  //Aloca memória na GPU
    cudaMalloc((void**)&d_a,nBytes);
    cudaMalloc((void**)&d_b,nBytes);
    cudaMalloc((void**)&d_c,nBytes);

  //Copia dados da CPU para a GPU
    cudaMemcpy(d_a, h_a, nBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, nBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_c, h_c, nBytes, cudaMemcpyHostToDevice);

  //Declaração do tamanho do grid e tamanho dos blocos
    dim3 blockSize(2,2);
    dim3 gridSize(N/2,N/2);

    //Executa o kernel
    additionMatricesKernel<<<gridSize,blockSize>>>(d_a, d_b, d_c);

  //Copia os dados de volta para a CPU
    (cudaMemcpy(h_c, d_c, nBytes, cudaMemcpyDeviceToHost));

  //Testa se a soma foi executada corretamente
    for(int i=0; i < N; i++){
        for(int j=0; j < N; j++){
            assert(h_a[i][j] + h_b[i][j] == h_c[i][j]);
        }
    }
  printf("\nSOMA EXECUTADA COM SUCESSO!\n\n");

  //Desaloca memória
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
}


SOMA EXECUTADA COM SUCESSO!




In [18]:
%%nvprof
#include <stdio.h>
#include <assert.h>

#define N 16

__global__ void additionMatricesKernel(int *d_a, int *d_b, int *d_c){
  //Define os índices das threads
  int i = threadIdx.y+blockIdx.y*blockDim.y;
  int j = threadIdx.x+blockIdx.x*blockDim.x;
  //Realiza a soma de cada elemento
  d_c[i*N+j] = d_a[i*N+j] + d_b[i*N+j];
}

int main(){
  //Define as matrizes quadradas NxN
    int h_a[N][N], h_b[N][N], h_c[N][N];

  //Preenche as matrizes
    for(int i = 0; i < N; i++){
        for(int j = 0; j < N; j++){
            h_a[i][j] = h_b[i][j] = i+j;
            h_c[i][j] = 0;
        }
    }

    //Declara os ponteiros para a GPU
    int *d_a, *d_b, *d_c;

    const int nBytes = N * N * sizeof(int);

  //Aloca memória na GPU
    cudaMalloc((void**)&d_a,nBytes);
    cudaMalloc((void**)&d_b,nBytes);
    cudaMalloc((void**)&d_c,nBytes);

  //Copia dados da CPU para a GPU
    cudaMemcpy(d_a, h_a, nBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, nBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_c, h_c, nBytes, cudaMemcpyHostToDevice);

  //Declaração do tamanho do grid e tamanho dos blocos
    dim3 blockSize(2,2);
    dim3 gridSize(N/2,N/2);

    //Executa o kernel
    additionMatricesKernel<<<gridSize,blockSize>>>(d_a, d_b, d_c);

  //Copia os dados de volta para a CPU
    (cudaMemcpy(h_c, d_c, nBytes, cudaMemcpyDeviceToHost));

  //Testa se a soma foi executada corretamente
    for(int i=0; i < N; i++){
        for(int j=0; j < N; j++){
            assert(h_a[i][j] + h_b[i][j] == h_c[i][j]);
        }
    }
  printf("\nSOMA EXECUTADA COM SUCESSO!\n\n");

  //Desaloca memória
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
}

==11962== NVPROF is profiling process 11962, command: /content/code.out

SOMA EXECUTADA COM SUCESSO!

==11962== Profiling application: /content/code.out
==11962== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   40.31%  3.3280us         1  3.3280us  3.3280us  3.3280us  additionMatricesKernel(int*, int*, int*)
                   32.57%  2.6890us         3     896ns     736ns  1.1840us  [CUDA memcpy HtoD]
                   27.12%  2.2390us         1  2.2390us  2.2390us  2.2390us  [CUDA memcpy DtoH]
      API calls:   83.51%  188.02ms         3  62.672ms  3.3260us  188.01ms  cudaMalloc
                   16.32%  36.751ms         1  36.751ms  36.751ms  36.751ms  cudaLaunchKernel
                    0.07%  160.79us       114  1.4100us     149ns  59.684us  cuDeviceGetAttribute
                    0.05%  115.74us         3  38.579us  2.6910us  106.01us  cudaFree
                    0.03%  62.588us         4  15.647us  7.83

In [20]:
%%gpu
#include <cuda_runtime.h>
#include <stdlib.h>
#include <stdio.h>
#include <assert.h>
__shared__ int k;
__global__ void crivo_array(int *a)

{

	int idx = blockIdx.x*blockDim.x+threadIdx.x; // parassar entre as threads e blocls de threads
	a[idx]= idx;
	k=2;


	while(k*2<=idx){

		if (((a[idx] % k) == (0))&&(a[idx] > 0)&&(a[idx] != k)){ //verifico se é multipo de k
			a[idx] = a[idx] * -1; //marco o elemento na lista
		}

		k++;

		while(a[k]<0){
			k++;
		}

	}

}



 int main(void)
{

	int *a_h, *a_d, threads, blocks;

	const int N=1000; //Numero de elementos da lista


	//Determino o numero de threads e blocks de threads usados
	if (N < 256)
	{
		threads = N;
		blocks = 1;
	}
	else
	{
		threads = 256;
		if ((N % threads) == 0)
			blocks = N/threads;
		else
			blocks = (N/threads) + 1;
	}
	//Fim do determincao

	size_t size = N*sizeof(int); //determinando o tamanho do vetor de inteiros

	a_h=(int*)malloc(size); //alocando espaço na memoria da CPU

	cudaMalloc((void**)&a_d,size); //alocando espaço na memoria da GPU

	unsigned int timer = 0;

	//faz crivo
	cudaMemcpy(a_d,a_h,size,cudaMemcpyHostToDevice);
	crivo_array<<<blocks,threads>>>(a_d);
	cudaMemcpy(a_h,a_d,size,cudaMemcpyDeviceToHost);
	//fim do crivo

	printf("São primos os numeros entre 2 e %d\n", N);
/*	for(int i=2; i<N; i++)
		if (a_h[i] > 0)
			printf("%d\n",a_h[i]); //imprimo somente os numeros não marcados (primos)
*/

	free(a_h); // Liberando memoria da CPU
	cudaFree(a_d); // Liberando memoria da GPU
	return 0;
}

São primos os numeros entre 2 e 1000



In [21]:
%%nvprof
#include <cuda_runtime.h>
#include <stdlib.h>
#include <stdio.h>
#include <assert.h>
__shared__ int k;
__global__ void crivo_array(int *a)

{

	int idx = blockIdx.x*blockDim.x+threadIdx.x; // parassar entre as threads e blocls de threads
	a[idx]= idx;
	k=2;


	while(k*2<=idx){

		if (((a[idx] % k) == (0))&&(a[idx] > 0)&&(a[idx] != k)){ //verifico se é multipo de k
			a[idx] = a[idx] * -1; //marco o elemento na lista
		}

		k++;

		while(a[k]<0){
			k++;
		}

	}

}



 int main(void)
{

	int *a_h, *a_d, threads, blocks;

	const int N=1000; //Numero de elementos da lista


	//Determino o numero de threads e blocks de threads usados
	if (N < 256)
	{
		threads = N;
		blocks = 1;
	}
	else
	{
		threads = 256;
		if ((N % threads) == 0)
			blocks = N/threads;
		else
			blocks = (N/threads) + 1;
	}
	//Fim do determincao

	size_t size = N*sizeof(int); //determinando o tamanho do vetor de inteiros

	a_h=(int*)malloc(size); //alocando espaço na memoria da CPU

	cudaMalloc((void**)&a_d,size); //alocando espaço na memoria da GPU

	unsigned int timer = 0;

	//faz crivo
	cudaMemcpy(a_d,a_h,size,cudaMemcpyHostToDevice);
	crivo_array<<<blocks,threads>>>(a_d);
	cudaMemcpy(a_h,a_d,size,cudaMemcpyDeviceToHost);
	//fim do crivo

	printf("São primos os numeros entre 2 e %d\n", N);
/*	for(int i=2; i<N; i++)
		if (a_h[i] > 0)
			printf("%d\n",a_h[i]); //imprimo somente os numeros não marcados (primos)
*/

	free(a_h); // Liberando memoria da CPU
	cudaFree(a_d); // Liberando memoria da GPU
	return 0;
}

==12738== NVPROF is profiling process 12738, command: /content/code.out
São primos os numeros entre 2 e 1000
==12738== Profiling application: /content/code.out
==12738== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   96.66%  115.77us         1  115.77us  115.77us  115.77us  crivo_array(int*)
                    2.16%  2.5920us         1  2.5920us  2.5920us  2.5920us  [CUDA memcpy DtoH]
                    1.18%  1.4080us         1  1.4080us  1.4080us  1.4080us  [CUDA memcpy HtoD]
      API calls:   99.63%  192.76ms         1  192.76ms  192.76ms  192.76ms  cudaMalloc
                    0.11%  203.50us         2  101.75us  71.840us  131.66us  cudaMemcpy
                    0.10%  202.01us         1  202.01us  202.01us  202.01us  cudaLaunchKernel
                    0.08%  152.36us       114  1.3360us     142ns  56.324us  cuDeviceGetAttribute
                    0.07%  128.04us         1  128.04us  128.04us  128.04us